In [1]:
import psycopg2
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
conn = psycopg2.connect("dbname=oakland user=danaezoule")
cur = conn.cursor()
cur.execute("SELECT * FROM area_features;")
df = pd.DataFrame(cur.fetchall())
cdf = df.copy()
cdf.columns = ['Group_Block', 'Year', 'Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']

In [30]:
# Splitting data by year, clustering by first year, predicting clusters for following years.
# Simple clustering model - first pass - on count on crimes in each category.
from sklearn.cluster import KMeans
km = KMeans()
columns = ['Quality', 'Nonviolent', 'Vehicle_Break_In', 'Vehicle_Theft', 'Violent']
clus = km.fit_predict(cdf[cdf.Year == 2009].sort('Group_Block')[columns])
clus10 = km.predict(cdf[cdf.Year == 2010].sort('Group_Block')[columns])
clus11 = km.predict(cdf[cdf.Year == 2011].sort('Group_Block')[columns])

In [ ]:
'''Feature Engineering Brainstorming:
Normalization:
    Normalize by population: none, assume census divisions cover this
    Normalize by geography (square footage or meterage)
    Normalize by total crime count

Geographical:
    Census tracts, group blocks, or blocks

Time Group By:
    Month, quarter, year

Time features:
    Count for weekday or weekend
    Count for time of day (morning, afternoon, eve, night)
        Split by data. First hypothesis:
        Morning: 6am-noon
        Afternoon: noon-6pm
        Eve: 6pm-midnight
        Early: midnight-6am
    
Housing etc:
    Are Trulia neighborhoods census tracts? Can I get block group info from Trulia?
    Will the ACS be helpful? Can I get yearly or quarterly ACS information?
    
Time component:
    Create centroids from earliest data, map all points to same centroids
    Create new centroids for each year (with varied data) as below 
    
Data that varies from year to year:
    If I use it, can I detect similar centroids between years?
    Should I instead ignore this completely, despite losing Lovely connection?
'''

In [16]:
conn = psycopg2.connect("dbname=oakland user=danaezoule")
cur = conn.cursor()
cur.execute("SELECT * FROM crime;")
df = pd.DataFrame(cur.fetchall())
cdf = df.copy()
cdf.columns = cdf.columns = ['Idx', 'OPD_RD', 'Date', 'Time', 'Lat', 'Lng', 'year', 'year_month', 'quality', 'nonviolent', 'car_break_in', 'car_theft', 'violent', 'geom']
cdf['day_of_week'] = pd.DatetimeIndex(cdf.Date).dayofweek
cdf['hour'] = [i.hour for i in cdf.Time]

In [23]:
# look for outliers with respect to hour, day of week
# lat/long outliers are controlled for by only selecting areas with crime counts with min 1/year
cdf.describe()

,Idx,Lat,Lng,year,year_month,quality,nonviolent,car_break_in,car_theft,violent,day_of_week,hour
count,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000,224403.000000
mean,478469.990245,37.790798,-122.224579,2011.608267,2011614.733471,0.170439,0.282594,0.138733,0.165738,0.242497,2.968423,12.821157
std,133396.127438,0.094039,0.366965,1.743823,1743.642339,0.376019,0.450262,0.345668,0.371846,0.428594,1.977916,7.158425
min,2.000000,0.000000,-122.672055,2009.000000,2009001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,369538.000000,37.768153,-122.264240,2010.000000,2010007.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,8.000000
50%,487141.000000,37.794056,-122.226953,2012.000000,2012003.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,14.000000
75%,599825.500000,37.811339,-122.193836,2013.000000,2013007.000000,0.000000,1.000000,0.000000,0.000000,0.000000,5.000000,19.000000
max,679442.000000,48.159570,0.000000,2015.000000,2015012.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,23.000000


In [44]:
'''
cdf.columns = ['Idx', 'OPD_RD', 'Date', 'Time', 'Lat', 'Lng', 'year', 'year_month', 'quality', 'nonviolent', 'car_break_in', 'car_theft', 'violent', 'geom', 'block_group']
#cdf['quarter'] = pd.DatetimeIndex(cdf.Date).quarter
cdf['day_of_week'] = pd.DatetimeIndex(cdf.Date).dayofweek
#cdf['week'] = pd.DatetimeIndex(cdf.Date).week
#cdf['month'] = pd.DatetimeIndex(cdf.Date).month
#cdf['day'] = pd.DatetimeIndex(cdf.Date).day
#cdf['day_of_year'] = pd.DatetimeIndex(cdf.Date).dayofyear
#cdf['week_of_year'] = pd.DatetimeIndex(cdf.Date).weekofyear
cdf['hour'] = [i.hour for i in cdf.Time]
#combine date and time column so I can plot over 'em
#cdf['Datetime'] = pd.to_datetime(cdf['Date'].astype(str) + ' ' + cdf['Time'].astype(str))
'''